# 강좌 엑셀 파일에서 불러오기

In [15]:
# from urllib.request import urlopen
# html = urlopen("https://www.frip.co.kr/daily/learning/all?orderBy=popular&page=1")  
# bsObject = BeautifulSoup(html, "html.parser")
# print(bsObject)

In [39]:
import requests
from bs4 import BeautifulSoup

headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}
url = "https://www.frip.co.kr/products/108923"
webpage = requests.get(url, headers=headers)
soup = BeautifulSoup(webpage.content, "html.parser")
soup.select(
'body')
# print(soup)

[<body><noscript>If you're seeing this message, that means <strong>JavaScript has been disabled on your browser</strong>, please <strong>enable JS</strong> to make this app work.</noscript><div id="app"></div><div id="modal-root"></div><div id="dnd-root"></div><link href="https://fonts.googleapis.com/css?family=Noto+Sans+KR:300,400,500,700" rel="stylesheet"/><script src="/runtime~main.bcf9f3a8eff7fa7c2423.js" type="text/javascript"></script><script src="/vendors~main.3749da4eac8b61e1a652.chunk.js" type="text/javascript"></script><script src="/main.680bb431ba4e89ad202b.chunk.js" type="text/javascript"></script></body>]

In [8]:
with open('lecture.csv') as file:
    csv_data = []
    for line in file.readlines():
        csv_data.append(line.split(','))
print(csv_data[0][0])

강좌명


In [ ]:
import numpy
csv_data = numpy.loadtxt('lecture.csv', delimiter=',')


In [ ]:
import pandas as pd
csv_data = pd.read_csv("./lecture.csv")
print(csv_data)

# 셀리니움을 활용하여 frip에서 크롤링

In [42]:
pip install selenium

  Using cached selenium-3.141.0-py2.py3-none-any.whl (904 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
conda install -c conda-forge selenium

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\sehyu\anaconda3

  added / updated specs:
    - selenium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.8.3                |   py37hc8dfbb8_1         3.1 MB  conda-forge
    python_abi-3.7             |          1_cp37m           4 KB  conda-forge
    selenium-3.141.0           |py37h8055547_1001         937 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will be INSTALLED:

  python_abi         conda-forge/win-64::python_abi-3.7-1_cp37m
  selenium           conda-forge/win-64::selenium-3.141.0-py37h8055547_1001

The following packages will be UPDATED:

  conda                       pkgs/main::conda-4.8.2-py37_0 --> conda-forge::conda-4.8.3-py37hc8dfbb8_1




p



==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base -c defaults conda




In [45]:
import time
import csv
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from datetime import datetime

driver = webdriver.Chrome('./chromedriver')

## 페이지 넘어가면서 크롤링

In [87]:
links = []
for i in range(1, 179):
    driver.get('https://www.frip.co.kr/daily/learning/all?page='+ str(i))
    time.sleep(1)
    lectures = driver.find_elements_by_class_name("sc-kGXeez")
    for lecture in lectures:
        a_name = lecture.find_element_by_tag_name('a').get_attribute('href')
        links.append(a_name)

# 크롤링한 페이지 url csv파일로 저장

## DataFrame 방식으로 저장

In [106]:
import pandas as pd

dataframe = pd.DataFrame(links)
dataframe.to_csv('./frip.csv', header=False,index=False)

## csv writer 방식으로 저장

In [4]:
import csv

In [150]:
f = open('./frip_csv.csv', 'w', newline='')
wr = csv.writer(f)
for link in links:
    wr.writerow([link])
f.close()

# 페이지 url이 저장된 csv 파일 불러오기

In [5]:
f = open('./frip.csv', 'r')
rdr = csv.reader(f)
new_links = []
for line in rdr:
    new_links.append(line[0])
f.close()

In [151]:
f = open('./frip_csv.csv', 'r')
rdr = csv.reader(f)
new_links = []
for line in rdr:
    new_links.append(line[0])
f.close()

In [86]:
new_links[1]

'https://www.frip.co.kr/products/108923'

# 페이지 별로 돌아가면서 데이터 크롤링

## 한개만 테스트

In [197]:
driver.get("https://www.frip.co.kr/products/114211")
time.sleep(1)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
meeting_id = "https://www.frip.co.kr/products/114211".split('/')[-1]
uid = driver.find_element_by_class_name('nCgAM').find_element_by_tag_name('a').get_attribute('href')
writer = soup.select_one('span.gYdgbL')
created_at = datetime.now()
updated_at = datetime.now()
is_period = 0
meeting_date = datetime.now()
temp = soup.select('section.sc-dUcZlc > div.sc-ZUflv > div.sc-esoVGF')
if temp[1].text[0].isdigit():
    temp_date = ''
    for t in temp[1].text:
        if t == '외':
            break
        if t in '(월화수목금토일요일년)':
            continue
        if t == ' ':
            t = '-'
        temp_date += t
    temp_date = temp_date[:-1]
    meeting_date = datetime.strptime(temp_date, '%Y-%m-%d-%H:%M')
period_date = ''
is_class = 1
max_person = 20
now_person = 1
content = '[내용]' + soup.select('div.cSNowP')[0].text
ref_url = new_links[0]
address = soup.select_one('div.sc-kRCAcj').text[:-5]
fee = soup.select_one('div.sc-eBipZS').text[:-1].replace(',', '')
unit = '원'
is_active = 1
like_temp = ''
for t in temp[0].text:
    if not t.isdigit():
        break
    like_temp += t
like_cnt = int(like_temp)
view_cnt = 0
score = 0
if temp[3].text[6].isdigit():
    score = float(temp[3].text[6:])
main_category = 0
tags = ''

In [175]:
meetings.append(
    [
        meeting_id,
        uid,
        writer,
        created_at,
        updated_at,
        is_period,
        meeting_date,
        period_date,
        is_class,
        max_person,
        now_person,
        content,
        ref_url,
        address,
        fee,
        unit,
        is_active,
        like_cnt,
        view_cnt,
        score,
        main_category,
        tags
    ]
)

## 미팅 시간 연산

In [85]:
from datetime import datetime

temp_date = ''
for t in temp[1].text:
    if t == '외':
        break
    if t in '(월화수목금토일요일년)':
        continue
    if t == ' ':
        t = '-'
    temp_date += t
temp_date = temp_date[:-1]
datetime_object = datetime.strptime(temp_date, '%Y-%m-%d-%H:%M')
print(datetime_object)

2020-05-23 13:00:00


## columns 생성

In [225]:
import pandas as pd
from datetime import datetime

meeting_columns = (
    "meeting_id",
    "uid",
    "writer",
    "created_at",
    "updated_at",
    "is_period",
    "meeting_date",
    "period_date",
    "is_class",
    "max_person",
    "now_person",
    "content",
    "ref_url",
    "address",
    "fee",
    "unit",
    "is_active",
    "like_cnt",
    "view_cnt",
    "score",
    "main_category",
    "tags"
)

image_columns = (
    "meeting_id",
    "image_url"
)

## 제대로 테스트

In [226]:
meetings = []
images = []

In [219]:
for link in range(len(new_links)):
    print(new_links[link])
    driver.get(new_links[link])
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    meeting_id = new_links[link].split('/')[-1]
    uid = driver.find_element_by_class_name('nCgAM').find_element_by_tag_name('a').get_attribute('href')
    writer = soup.select_one('span.gYdgbL')[1:-1]
    created_at = datetime.now()
    updated_at = datetime.now()
    is_period = 0
    meeting_date = datetime.now()
    temp = soup.select('section.sc-dUcZlc > div.sc-ZUflv > div.sc-esoVGF')
    if temp[-3].text == '':
        continue
    elif temp[-3].text[0].isdigit():
        temp_date = ''
        for t in temp[1].text:
            if t == '외':
                break
            if t in '(월화수목금토일요일년)':
                continue
            if t == ' ':
                t = '-'
            temp_date += t
        temp_date = temp_date[:-1]
        meeting_date = datetime.strptime(temp_date, '%Y-%m-%d-%H:%M')
    period_date = ''
    is_class = 1
    max_person = 20
    now_person = 1
    content = '[내용]' + soup.select('div.cSNowP')[0].text
    ref_url = new_links[link]
    print(soup.select_one('div.sc-kRCAcj').text[:-5])
    address = soup.select_one('div.sc-kRCAcj').text[:-5]
    fee = soup.select_one('div.sc-eBipZS').text[:-1].replace(',', '')
    unit = '원'
    is_active = 1
    like_temp = ''
    for t in temp[0].text:
        if not t.isdigit():
            break
        like_temp += t
    like_cnt = int(like_temp)
    view_cnt = 0
    score = 0
    if temp[-1].text[6].isdigit():
        score = float(temp[-1].text[6:])
    main_category = 0
    tags = ''
    
    # 이미지 저장
    images_link = driver.find_elements_by_class_name('cljRKC')
    image_urls = set()
    for i in images_link:
        image_urls.add(i.get_attribute('src'))
    image_url = ' '.join(image_urls)
    
    # 데이터 저장
    meetings.append(
    [
        meeting_id,
        uid,
        writer,
        created_at,
        updated_at,
        is_period,
        meeting_date,
        period_date,
        is_class,
        max_person,
        now_person,
        content,
        ref_url,
        address,
        fee,
        unit,
        is_active,
        like_cnt,
        view_cnt,
        score,
        main_category,
        tags
    ]
        
    images.append(
    [
        meeting_id,
        image_url
    ]
    )
)

https://www.frip.co.kr/products/132118
https://www.frip.co.kr/products/112983
강원 고성군 죽왕면 자작도선사길 86
https://www.frip.co.kr/products/135449
경기 성남시 분당구 정자동 85-18 1층, 무드문
https://www.frip.co.kr/products/117716
서울 서초구 동광로12가길 16 1층 플라워모리아
https://www.frip.co.kr/products/135609
서울 마포구 동교로24길 17 3층 이코노미스타
https://www.frip.co.kr/products/121301
서울시 광진구 능동로 41길 42 1층 슬로우 허그
https://www.frip.co.kr/products/121429
서울시 중랑구 동일로 775 (중화3동 그린프라자 B1) 중화스쿼시헬스클럽
https://www.frip.co.kr/products/135704
서울 송파구 잠실동 175-14 10층 토즈 스터디 모임공간
https://www.frip.co.kr/products/134337
온라인에서 진행
https://www.frip.co.kr/products/112233
https://www.frip.co.kr/products/131559
서울시 마포구 동교로 32길 7, 4층
https://www.frip.co.kr/products/135965
서울시 동작구 노량진로 48, 현대아파트상가 나동 2층 203호 (롤앤쿠키공방)
https://www.frip.co.kr/products/126801
서울시 성북구 동소문로 21-20, 1층 아브릴 베이킹 스튜디오
https://www.frip.co.kr/products/126723
서울 마포구 합정동 395-1 3층
https://www.frip.co.kr/products/131981
서울시 성북구 성북로8길 4, 4층
https://www.frip.co.kr/products/134856
온라인에서 진행
https

서울시 마포구 공덕동 232-10 3층
https://www.frip.co.kr/products/108267
서울시 마포구 서교동 327-30
https://www.frip.co.kr/products/109370
서울시 관악구 낙성대역 3번 출구 동림오피스텔 b3
https://www.frip.co.kr/products/116406
서울시 성북구 동소문로 17길 29-2, 동소문피스커피 / 경기도 의정부시 의정로46번길 18 1층, 커피클래스
https://www.frip.co.kr/products/130521
서울시 강남구 신사동 550-16 지하1층  커피랑도서관 압구정점
https://www.frip.co.kr/products/134432
배송상품
https://www.frip.co.kr/products/133724
온라인
https://www.frip.co.kr/products/127953
서울시 마포구 백범로 199, 메트로디오빌
https://www.frip.co.kr/products/121946
서울시 중구 한강대로 416, 13층
https://www.frip.co.kr/products/117638
서울시 강남구 논현동 13-18번지 4층 [잇플]
https://www.frip.co.kr/products/120074
서울시 서초구 사평대로 368, 1층 투썸플레이스
https://www.frip.co.kr/products/134189
배송상품
https://www.frip.co.kr/products/109051
서울시 마포구 서교동442-1 성광빌딩201호
https://www.frip.co.kr/products/109037
서울 동작구 동작대로 49 호암빌딩 지하1층
https://www.frip.co.kr/products/134388
배송상품
https://www.frip.co.kr/products/130485
서울시 강남구 신사동 639-6 프랑소와펫
https://www.frip.co.kr/products/121309
서울시 광진구 능동로

서울 서초구 서운로 160 204호 (1층 파리바게트 건물)
https://www.frip.co.kr/products/115610
서울특별시 금천구 가산동 234-46 삼익상가 2층 하나로탁구클럽
https://www.frip.co.kr/products/120308
강원 양양군 현남면 새나루길 43, 1층 서프오션
https://www.frip.co.kr/products/115338
서울시 광진구 구의동 257-126, 3층
https://www.frip.co.kr/products/135053
헤이그라운드 서울숲점 10층
https://www.frip.co.kr/products/132963
서울시 강동구 암사동 504-29, 지하 1층 우리드리공방
https://www.frip.co.kr/products/130471
서울시 성동구 성수이로12길 6, 2층
https://www.frip.co.kr/products/118806
서울시 성동구 성수동1가 718, 트리마제
https://www.frip.co.kr/products/134282
온라인에서 진행
https://www.frip.co.kr/products/133599
서울시 서초구 신반포로23길 41, 잠원쇼핑센터 3층 314호
https://www.frip.co.kr/products/116614
서울시 강남구 대치동 919-35, 102호  벨로르플라워
https://www.frip.co.kr/products/115543
서울시 양천구 공항대로 596 호용빌딩지층
https://www.frip.co.kr/products/133885
서울 마포구 모래내로3길 11 상암미르웰오피스텔 아담한 홈공방
https://www.frip.co.kr/products/130789
서울시 마포구 연남동 249-7, 뒷문 오후의 작업실
https://www.frip.co.kr/products/131855
서울시 동대문구 천호대로83길 46, 4층 공방
https://www.frip.co.kr/products/112921
서울시 

서울시 금천구 탑골로 47 1층 원바이올린
https://www.frip.co.kr/products/119267
서울시 서대문구 신촌로 227, 3층 아트스페이스 (2호선 아현역 근처)
https://www.frip.co.kr/products/133010
서울시 종로구 자하문로5길 18-13, 1층 공유카페
https://www.frip.co.kr/products/131696
서울시 서초구 바우뫼로 220, 지층 보울하우스 아카데미
https://www.frip.co.kr/products/121502
서울시 중랑구 용마산로 615, 1층
https://www.frip.co.kr/products/109067
서울시 서초구 양재동 364번지 1층 스튜디오벨
https://www.frip.co.kr/products/123090
서울 광진구 아차산로 243, 건대입구역
https://www.frip.co.kr/products/115571
서울시 강남구 논현동 164-14, 3층 GSM테라스(신논현역 3번출구 5분거리)
https://www.frip.co.kr/products/135441
온라인
https://www.frip.co.kr/products/135249
우리 집!
https://www.frip.co.kr/products/135452
서울 마포구 중동 82-18 1층 아름다운미술마을
https://www.frip.co.kr/products/130505
서울시 강서구 화곡동 1053-4, 1층 도로시
https://www.frip.co.kr/products/128084
온라인으로 진행
https://www.frip.co.kr/products/111992
서울시 마포구 대흥동 376 단독건물2층 브런치스튜디오
https://www.frip.co.kr/products/108362
서울시 송파구 방이동 156-4 1층 104호
https://www.frip.co.kr/products/116625
서울시 양천구 신정동 329 목동신시가지아파트 14단지 A상가 3층 우수

ValueError: invalid literal for int() with base 10: ''

In [223]:
meeting_frame = pd.DataFrame(data=meetings, columns=meeting_columns)

In [224]:
meeting_frame.tail(100)

,meeting_id,uid,writer,created_at,updated_at,is_period,meeting_date,period_date,is_class,max_person,...,ref_url,address,fee,unit,is_active,like_cnt,view_cnt,score,main_category,tags
490,84280,https://www.frip.co.kr/hosts/4255,[디세],2020-05-22 17:23:22.542698,2020-05-22 17:23:22.542698,0,2020-05-23 14:00:00.000000,,1,20,...,https://www.frip.co.kr/products/84280,"서울시 노원구 공릉로34길 121, 3층 디세",35000,원,1,1779,0,4.92,0,
491,132249,https://www.frip.co.kr/hosts/9619,[초월 chowol],2020-05-22 17:23:28.464697,2020-05-22 17:23:28.464697,0,2020-05-22 17:23:28.464697,,1,20,...,https://www.frip.co.kr/products/132249,"서울특별시 서초구 서운로 160, 204호",65000,원,1,261,0,4.75,0,
492,136105,https://www.frip.co.kr/hosts/11554,[BODACAKE],2020-05-22 17:23:31.342696,2020-05-22 17:23:31.342696,0,2020-05-22 17:23:31.342696,,1,20,...,https://www.frip.co.kr/products/136105,서울시 서대문구 가재울미래로 15 A동 (상세주소 추후 안내),58000,원,1,7,0,0.00,0,
493,135331,https://www.frip.co.kr/hosts/9213,[레브센트],2020-05-22 17:23:35.748696,2020-05-22 17:23:35.748696,0,2020-05-22 17:23:35.748696,,1,20,...,https://www.frip.co.kr/products/135331,서울 성동구 마조로15가길 2 이엘아트빌 1층,75000,원,1,17,0,0.00,0,
494,135365,https://www.frip.co.kr/hosts/1097,[작업실 보통날],2020-05-22 17:23:39.789697,2020-05-22 17:23:39.789697,0,2020-05-22 17:23:39.789697,,1,20,...,https://www.frip.co.kr/products/135365,"경기도 성남시 분당구 느티로69번길 15, 작업실보통날",45000,원,1,17,0,0.00,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
585,124285,https://www.frip.co.kr/hosts/8082,[카이카스튜디오],2020-05-22 17:29:50.013223,2020-05-22 17:29:50.013223,0,2020-05-22 17:29:50.013223,,1,20,...,https://www.frip.co.kr/products/124285,"서울시 서초구 주흥길 77, 1층 Kaika Studio (논현역 도보 5분)",39000,원,1,290,0,4.96,0,
586,120252,https://www.frip.co.kr/hosts/6322,[민트],2020-05-22 17:29:54.567225,2020-05-22 17:29:54.567225,0,2020-05-22 17:29:54.567225,,1,20,...,https://www.frip.co.kr/products/120252,서울시 양천구 목동동로 53 (상세 주소 추후 공지),59000,원,1,871,0,4.96,0,
587,115574,https://www.frip.co.kr/hosts/238,[루나루-한강패들클럽],2020-05-22 17:29:58.153225,2020-05-22 17:29:58.153225,0,2020-05-22 17:29:58.153225,,1,20,...,https://www.frip.co.kr/products/115574,서울 광진구 자양동 94 서울윈드서핑장 47호 루나루,30000,원,1,1173,0,4.78,0,
588,115726,https://www.frip.co.kr/hosts/238,[루나루-한강패들클럽],2020-05-22 17:30:01.755224,2020-05-22 17:30:01.755224,0,2020-05-22 17:30:01.755224,,1,20,...,https://www.frip.co.kr/products/115726,서울 광진구 자양동 94 서울윈드서핑장 47호 루나루,30000,원,1,714,0,4.77,0,
